In [12]:
!pip install yfinance
#https://github.com/ranaroussi/yfinance
#https://finance.yahoo.com/

In [13]:
import yfinance as yf
import numpy as np
import sys
import pandas as pd
import requests 
from pandas_datareader import data as pdr
pd.options.display.max_columns = None

In [14]:
def mergedataframes(df1 , df2, stringdf1 = None, stringdf2 = None):
  dfmerged = pd.merge(df1, df2, on="Date",suffixes = (stringdf1,stringdf2))
  return dfmerged

In [15]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
url = "http://www.fundamentus.com.br/detalhes.php?papel="

html = requests.get(url,headers=headers)
tickers = pd.read_html(html.content)[0]

tickers

,Papel,Nome Comercial,Razão Social
0,AALR3,ALLIAR,CENTRO DE IMAGEM DIAGNOSTICOS S.A.
1,ABCB3,ABC Brasil,BANCO ABC BRASIL S/A
2,ABCB4,ABC Brasil,BANCO ABC BRASIL S/A
3,ABEV3,AMBEV S/A,AMBEV S.A.
4,ABRE3,SOMOS EDUCA,SOMOS EDUCAÇÃO S.A.
...,...,...,...
1017,WLMM3,WLM,WLM INDÚSTRIA E COMÉRCIO S/A
1018,WLMM4,WLM,WLM INDÚSTRIA E COMÉRCIO S/A
1019,WMBY3,WEMBLEY SOCIEDADE ANÔNIMA,WEMBLEY SOCIEDADE ANÔNIMA
1020,WSON33,Wilson Sons,WILSON SONS LIMITED


#Single Processing

Aproximadamente, 12 minutos para extrair o histórico de todas as ações listadas na B3

In [16]:
Choose_Stocks = tickers["Papel"].values + ".SA"

df_final = pd.DataFrame()

for i in Choose_Stocks:
  Stock = yf.Ticker(i)
  yf.pdr_override()
  dfStock = Stock.history(period="max")
  dfStock["Papel"] = i
  df_final = df_final.append(dfStock)

df_final = df_final[df_final.columns[:-1]]
df_final

- ABCB3.SA: None
- ABRE3.SA: None
- ABYA3.SA: No data found, symbol may be delisted
- ACES3.SA: None
- ACES4.SA: None
- AEDU11.SA: None
- AEDU3.SA: None
- AESL3.SA: None
- AESL4.SA: None
- AFLU3.SA: None
- AFLU5.SA: None
- AGEI3.SA: No data found, symbol may be delisted
- AGEN33.SA: No data found, symbol may be delisted
- AGIN3.SA: No data found, symbol may be delisted
- ALBA3.SA: None
- ALLL11.SA: None
- ALLL3.SA: None
- ALLL4.SA: None
- ALSC3.SA: No data found, symbol may be delisted
- AMBV3.SA: None
- AMBV4.SA: None
- AMIL3.SA: None
- AMPI3.SA: None
- AORE3.SA: None
- APTI4.SA: No data found, symbol may be delisted
- ARCE3.SA: No data found, symbol may be delisted
- ARCZ3.SA: None
- ARCZ6.SA: None
- ARLA3.SA: None
- ARLA4.SA: None
- ARPS3.SA: No data found, symbol may be delisted
- ARPS4.SA: No data found, symbol may be delisted
- ARTE3.SA: None
- ARTE4.SA: None
- ARTR3.SA: None
- ASSM3.SA: None
- ASSM4.SA: None
- ASTA4.SA: None
- AURA32.SA: No data found, symbol may be delisted
- A

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Papel
Date,,,,,,,,
2016-10-28,19.106026,19.561404,18.729846,19.007032,6342600.0,0.0,0.0,AALR3.SA
2016-10-31,18.997131,19.007030,17.334015,17.878487,2523300.0,0.0,0.0,AALR3.SA
2016-11-01,17.878488,18.195272,16.987534,17.720097,996200.0,0.0,0.0,AALR3.SA
2016-11-03,17.819092,18.056680,17.136027,17.809193,621000.0,0.0,0.0,AALR3.SA
2016-11-04,17.769594,17.937886,17.532005,17.571604,389800.0,0.0,0.0,AALR3.SA
...,...,...,...,...,...,...,...,...
2021-12-13,23.969999,24.469999,23.639999,23.639999,1849000.0,0.0,0.0,YDUQ3.SA
2021-12-14,24.240000,24.309999,23.040001,23.059999,1852700.0,0.0,0.0,YDUQ3.SA
2021-12-15,23.049999,23.580000,22.700001,23.400000,1945900.0,0.0,0.0,YDUQ3.SA


#MultiProcessing

Aproximadamente, 55 segundos para extrair o histórico de todas as ações listadas na B3

In [17]:
def scrape_Yf_Finance(Choose_Stocks):

  Stock = yf.Ticker(Choose_Stocks)
  yf.pdr_override()
  dfStock = Stock.history(period="max")
  dfStock["Papel"] = Choose_Stocks
    
  return dfStock

In [19]:
from multiprocessing import Pool

Choose_Stocks = tickers["Papel"].values + ".SA"
p = Pool(50)
df_final = p.map(scrape_Yf_Finance, Choose_Stocks)
p.terminate()
p.join()

df_final = pd.concat(df_final)
df_final = df_final[df_final.columns[:-1]]

- ACES3.SA: None
- BNCA3.SA: No data found, symbol may be delisted
- BICB4.SA: None
- CRBM3.SA: None
- BPIA3.SA: None
- AORE3.SA: None
- BISA3.SA: None
- CLAN4.SA: No data found, symbol may be delisted
- CCPR3.SA: No data found, symbol may be delisted
- BVMF3.SA: None
- AMBV3.SA: None
- ACES4.SA: None
- ARLA3.SA: None
- BFIT3.SA: None
- CMET4.SA: None
- ALLL3.SA: None
- AFLU5.SA: None
- BELG3.SA: None
- ARTE4.SA: None
- CNTO3.SA: No data found, symbol may be delisted
- BSCT3.SA: None
- AVIL3.SA: None
- CRTP5.SA: None
- ASTA4.SA: None
- BAHI11.SA: No data found for this date range, symbol may be delisted
- CAFE3.SA: None
- CRBM7.SA: None
- BPNM3.SA: No data found, symbol may be delisted
- ARTR3.SA: None
- AMBV4.SA: None
- BSCT5.SA: None
- BELG4.SA: None
- ARLA4.SA: None
- BFIT4.SA: None
- AGEI3.SA: No data found, symbol may be delisted
- CRUZ3.SA: None
- ALLL4.SA: None
- CAFE4.SA: None
- BPNM4.SA: None
- BSCT6.SA: None
- AGEN33.SA: No data found, symbol may be delisted
- BEMA3.SA: None


In [20]:
df_final.to_csv("base.csv", sep = ";", decimal=',')